In [ ]:
#!python3 -m pip install torch
#!python3 -m pip install matplotlib
#!python3 -m pip install numpy

# Neural Network basics

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

# Some work with tensors

All Types of Tensor

In [ ]:
torch.HalfTensor    # 2 byte, floating point
torch.FloatTensor   # 4 byte, floating point
torch.DoubleTensor  # 8 byte, floating point

torch.ShortTensor   # 2 byte, integer, signed
torch.IntTensor     # 4 byte, integer, signed
torch.LongTensor    # 8 byte, integer, signed

torch.CharTensor    # 1 byte, integer, signed
torch.ByteTensor    # 1 byte, integer, unsigned

## Initialization of Tensor

torch.Tensor is an alias for the default tensor type (torch.FloatTensor).

A tensor can be constructed from a Python list or sequence using the torch.tensor() constructor:

In [ ]:
x1 = torch.tensor([[1., -1.], [1., -1.]])
x1

In [ ]:
x1.shape

In [ ]:
torch.zeros([2,4])

In [ ]:
torch.ones(5,2)

In [ ]:
x2 = torch.IntTensor(x1.shape)
x2

In [ ]:
torch.ones_like(x2)

In [ ]:
x3 = x1.zero_()
x3

In [ ]:
x1

In [ ]:
x2.zero_()
torch.allclose(x1, x2)

Initialization of distributions

In [ ]:
x = torch.randn((3, 4))   # normal distribution
x

In [ ]:
x.random_(0, 10)    # discrete (int) uniform distribution at [0, 10]
x

In [ ]:
x.uniform_(0, 10)    # uniform distribution at [0, 10]
x

In [ ]:
x.normal_(mean=0, std=1)  # normal with mean=0 and variance=1
x

In [ ]:
y = x.bernoulli(p=0.4)   # bernoulli with param p=0.4
y

# Operations with tensors

In [ ]:
x = torch.tensor([[1., 2.], [3., 4.]])
y = torch.tensor([[1., -1.], [-1., 1.]])

In [ ]:
x.log()

In [ ]:
y.sin()

In [ ]:
y[x == y]

In [ ]:
y[y < x]

## Sum(), Max(), Min(), Mean()

In [ ]:
print('Sum of elements in x:\n', x.sum())

## Item()

In [ ]:
print('Sum of elements in y:\n', y.sum().item())

## torch.add()

In [ ]:
print('Adding one tensor to another:\n', torch.add(x,y))

## torch.mul()

In [ ]:
print('Multiplying one tensor with the other one:\n', torch.mul(x,y))

## t() (transpose) and torch.matmul()

In [ ]:
input_shape = 3
weights = torch.rand([input_shape])
#weights = torch.tensor([1., -1., 1.])
print('weights', weights)

inp = torch.tensor([1., -1., 1.]).t()
print('input', inp)

torch.matmul(weights, inp)

## torch.cat()

In [ ]:
torch.cat((weights, torch.tensor([0.5])), 0)


## changing shapes

In [ ]:
b = torch.zeros(3, 2)
b

In [ ]:
b.stride()

In [ ]:
b.view(2, 3)

In [ ]:
b.T

In [ ]:
b.T.stride()

In [ ]:
b.T.is_contiguous()

In [ ]:
a = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [ ]:
a.shape

In [ ]:
a[:, :, None].shape

## type_as, to

In [ ]:
b.type_as(torch.IntTensor())
b.to(torch.int32)

## NumPy

In [ ]:
a = np.zeros((2,4))
t = torch.from_numpy(a)

In [ ]:
t -= torch.tensor([[4., 4., 4., 4.],
                   [4., 4., 4., 4.]])
print(t, a)

In [ ]:
b = t.numpy()
b

# Specifying device and type

In [ ]:
#device = torch.device('cuda:0')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

t = torch.ones([2, 4], dtype=torch.float64, device=device)

In [ ]:
t.is_cuda

In [ ]:
!nvidia-smi
t

In [ ]:
t = t.cpu()

In [ ]:
torch.cuda.empty_cache()
!nvidia-smi

In [ ]:
t = t.to(device)
t

In [ ]:
t1 = torch.tensor([[1., 2.], [3., 4.]]).cpu()
t2 = torch.tensor([[1., 2.], [3., 4.]]).cuda()

t1 + t2

In [ ]:
t1 = t1.cuda()

t1 + t2

# Max Pooling

In [ ]:
def max_pooling(image, kernel_size):
    '''
    Your code HERE
    '''
    return feature_map

image = torch.ByteTensor(3, 4, 4)
print(image)
feature_map = max_pooling(image, 2)
feature_map

# Vertical and Horisontal mean

In [ ]:
def horizontal_mean(t):
  return t.mean(dim=1)

In [ ]:
def vertical_mean(t):
  return t.mean(dim=0)

In [ ]:
def mean(t):
  return t.mean(dim=(0, 1))

In [ ]:
t = torch.zeros(3, 5)
t.uniform_(0, 10)
print(t)
horizontal_mean(t)
vertical_mean(t)
mean(t)

# Neuron model

## Threshold_function


$$\begin{equation}
        \begin{matrix}
        f(x) & =
        & \left\{
        \begin{matrix}
        1 & \mbox{if } x \geq threshold \\
        0 & \mbox{if } x \lt threshold
        \end{matrix} \right.
        \end{matrix}
    \end{equation}$$

In [ ]:
def threshold_function(x, threshold=0):
    return 0 if x < threshold else 1

In [ ]:
inp = [x*0.01 - 0.5 for x in range(100)]
plt.plot(inp, [threshold_function(x, threshold=0) for x in inp])

## Neuron model

In [ ]:
class Neuron(object):
    def __init__(self, input_shape=None, weights: torch.Tensor=None, activation_function=None, debug=False):

        assert input_shape is not None or weights is not None
        if weights is not None:
            self.weights = weights
        else:
            self.weights = torch.rand([input_shape, 1])

        self.activation_function = activation_function
        self.debug = debug

    def agregate_signal(self, input_tensor):
        if self.debug:
            print('Input_tensor:\n', input_tensor)
            print(f'Multiplying:{self.weights} * {input_tensor}')
        m = torch.mul(self.weights, input_tensor)
        if self.debug:
            print(f'Result:\n{m}')
        s = m.sum()
        if self.debug:
            print(f'Sum:\n{s}')
        output = s.item()
        if self.debug:
            print('Output without activation:\n', output)
        return output

    def activation(self, input_value):
        if self.activation_function is not None:
            output = self.activation_function(input_value)
        else:
            output = input_value
        if self.debug:
            print('Output after activation:\n', output)
        return output

    def forward(self, input_tensor):
        output = self.agregate_signal(input_tensor)
        output = self.activation(output)
        return output

# Linearly separable classes

## Dataset

In [ ]:
N = 10

C0_x0 = np.random.random(N)
C0_x1 = (C0_x0 + [np.random.randint(10)/10 for i in range(N)] + 0.1)
C0 = np.array([(C0_x0[i], C0_x1[i]) for i in range(len(C0_x0))])

C1_x0 = np.random.random(N)
C1_x1 = (C1_x0 - [np.random.randint(10)/10 for i in range(N)] - 0.1)
C1 = np.array([(C1_x0[i], C1_x1[i]) for i in range(len(C1_x0))])

# plotting
plt.scatter(C0_x0, C0_x1, s=10, c='red')
plt.scatter(C1_x0, C1_x1, s=10, c='blue')
plt.xlim([-0.1,1.1])
plt.ylim([-1.1,2.1])
plt.grid(True)

## Classification

$$\begin{equation}
        \begin{matrix}
        \left\{
        \begin{matrix}
        w_0 x_0 + w_1 x_1 \geq 0 & \mbox{C0 (red)} \\
        w_0 x_0 + w_1 x_1 \lt 0 & \mbox{C1 (blue)}
        \end{matrix} \right.
        \end{matrix}
    \end{equation}$$

$$w_0 x_0 + w_1 x_1 = 0$$

$$x_1 = - w_0 / w_1 * x_0$$

$$ k = - w_0 / w_1$$

In [ ]:
w0 = -1
w1 = 1

wf_x0 = [-1, 0 , 1, 2]
wf_x1 = [x0*(-w0/w1) for x0 in wf_x0]

# plotting
plt.plot(wf_x0, wf_x1)
plt.grid(True)
plt.xlim([-0.1,1.1])
plt.ylim([-1.1,2.1])
plt.scatter(C0_x0, C0_x1, s=10, c='red')
plt.scatter(C1_x0, C1_x1, s=10, c='blue')

In [ ]:
weights = torch.tensor([w0, w1])
nn = Neuron(weights=weights, activation_function=threshold_function, debug=True)

x = [0, 1]

out = nn.forward(torch.tensor(x))

# plotting
if out >= 0.5:
    print("Class C0 (Red)")
    plt.scatter(x[0], x[1], s=10, c='red')

else:
    print("Class C1 (Blue)")
    plt.scatter(x[0], x[1], s=10, c='blue')

plt.plot(wf_x0, wf_x1)
plt.grid(True)
plt.xlim([-0.1,1.1])
plt.ylim([-1.1,2.1])

# Bias

## Dataset

In [ ]:
N = 10
b = 2

C0_x0 = np.random.random(N)
C0_x1 = (C0_x0 + [np.random.randint(10)/10 for i in range(N)] + 0.1 + b)
C0 = np.array([(C0_x0[i], C0_x1[i]) for i in range(len(C0_x0))])

C1_x0 = np.random.random(N)
C1_x1 = (C1_x0 - [np.random.randint(10)/10 for i in range(N)] - 0.1 + b)
C1 = np.array([(C1_x0[i], C1_x1[i]) for i in range(len(C1_x0))])

# plotting
plt.scatter(C0_x0, C0_x1, s=10, c='red')
plt.scatter(C1_x0, C1_x1, s=10, c='blue')
plt.xlim([-0.1,1.1])
plt.ylim([-1.1+b,2.1+b])
plt.grid(True)

In [ ]:
w0 = -1
w1 = 1

wf_x0 = [-1, 0 , 1, 2]
wf_x1 = [x0*(-w0/w1) for x0 in wf_x0]

# plotting
plt.plot(wf_x0, wf_x1)
plt.grid(True)
plt.xlim([-0.1,1.1])
plt.ylim([-1.1,2.1+b])
plt.scatter(C0_x0, C0_x1, s=10, c='red')
plt.scatter(C1_x0, C1_x1, s=10, c='blue')

$$\begin{equation}
        \begin{matrix}
        \left\{
        \begin{matrix}
        w_0 x_0 + w_1 x_1 + bias \geq 0 & \mbox{C0 (red)} \\
        w_0 x_0 + w_1 x_1 + bias \lt 0 & \mbox{C1 (blue)}
        \end{matrix} \right.
        \end{matrix}
    \end{equation}$$

$$w_0 x_0 + w_1 x_1 + bias = 0$$

$$x_1 = - w_0 / w_1 * x_0 - bias/w_1 $$

$$ k = - w_0 / w_1, b = - bias / w_1$$

<img src="img/neuron_bias.png" width=1000>

In [ ]:
class BiasNeuron(Neuron):
    def __init__(self, input_shape=None, weights: torch.Tensor=None, bias=None, activation_function=None,
                     debug=False):
        super().__init__(input_shape, weights, activation_function, debug)

        if bias is not None:
            self.bias = bias


    def forward(self, input_tensor):
        output = self.agregate_signal(input_tensor)

        output = output + self.bias

        output = self.activation(output)
        return output

In [ ]:
w0 = -0.5
w1 = 0.5
b = 2.

bias = -b * w1

wf_x0 = [-1, 0 , 1, 2]
wf_x1 = [x0*(-w0/w1) + b for x0 in wf_x0]

# plotting
plt.plot(wf_x0, wf_x1)
plt.grid(True)
plt.xlim([-0.1,1.1])
plt.ylim([-1.1,2.1+b])
plt.scatter(C0_x0, C0_x1, s=10, c='red')
plt.scatter(C1_x0, C1_x1, s=10, c='blue')

In [ ]:
weights = torch.tensor([w0, w1])
nn = BiasNeuron(weights=weights, bias=bias, debug=True)

x = [0, 1]

out = nn.forward(torch.tensor(x))

# plotting
if out >= 0.5:
    print("Class C0 (Red)")
    plt.scatter(x[0], x[1], s=10, c='red')

else:
    print("Class C1(Blue)")
    plt.scatter(x[0], x[1], s=10, c='blue')

plt.plot(wf_x0, wf_x1)
plt.grid(True)
plt.xlim([-0.1,1.1])
plt.ylim([-1.1,2.1+b])

# XOR

In [ ]:
C0 = np.array([(0,1), (1,0)])

C1 = np.array([(0,0), (1,1)])


# plotting
plt.scatter([C0[i][0] for i in range(len(C0))],
            [C0[i][1] for i in range(len(C0))], s=10, c='red')
plt.scatter([C1[i][0] for i in range(len(C1))],
            [C1[i][1] for i in range(len(C1))], s=10, c='blue')
#plt.plot(f)
plt.grid(True)
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])

In [ ]:
w0 = 1
w1 = 1
b = 1.5

bias = -b * w1

C0 = np.array([(0,1), (1,0)])

C1 = np.array([(0,0), (1,1)])

# plotting
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figheight(4)
fig.set_figwidth(13)

wf_x0 = [-2, -1, 0 , 1, 2]
wf_x1 = [x0*(-w0/w1) + 1.5 for x0 in wf_x0]
ax1.plot(wf_x0, wf_x1)

wf_x0 = [-2, -1, 0 , 1, 2]
wf_x1 = [x0*(-w0/w1) + 0.5 for x0 in wf_x0]
ax2.plot(wf_x0, wf_x1)

for ax in [ax1, ax2]:
    ax.scatter([C0[i][0] for i in range(len(C0))],
                [C0[i][1] for i in range(len(C0))], s=10, c='red')
    ax.scatter([C1[i][0] for i in range(len(C1))],
                [C1[i][1] for i in range(len(C1))], s=10, c='blue')

    ax.grid(True)
    ax.set_xlim([-0.1,1.1])
    ax.set_ylim([-0.1,1.1])

In [ ]:
class Perceptron:
    def __init__(self):
        self.first_layer = []
        self.second_layer = []

    def forward(self, input_tensor):
        first_layer_output = []

        for neuron in self.first_layer:
            output = neuron.forward(input_tensor)
            first_layer_output.append(output)

        second_layer_input = torch.tensor(first_layer_output)

        for neuron in self.second_layer:
            output = neuron.forward(second_layer_input)

        return output

In [ ]:
w0 = 1
w1 = 1
b = 1.5

bias = -b * w1

wf_x0 = [-2, -1, 0 , 1, 2]
wf_x1 = [x0*(-w0/w1) - bias/w1 for x0 in wf_x0]
plt.plot(wf_x0, wf_x1)

weights = torch.tensor([w0, w1])
neuron = BiasNeuron(weights=weights, bias=bias, activation_function=threshold_function)

X = [(i/100, i/100) for i in range(100)]
for x in X:
    out = neuron.forward(torch.tensor(x))

    #print("out", out)

    if out >= 0.5:
        #print("Class C0")
        plt.scatter(x[0], x[1], s=10, c='red')

    else:
        #print("Class C1")
        plt.scatter(x[0], x[1], s=10, c='blue')

plt.grid(True)
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])

In [ ]:
nn = Perceptron()

w0 = 1
w1 = 1
b = 1.5

bias = -b * w1

weights = torch.tensor([w0, w1])
neuron = BiasNeuron(weights=weights, bias=bias, activation_function=threshold_function)

nn.first_layer.append(neuron)

w0 = 1
w1 = 1
b = 0.5

bias = -b * w1

weights = torch.tensor([w0, w1])
neuron = BiasNeuron(weights=weights, bias=bias, activation_function=threshold_function)

nn.first_layer.append(neuron)

w0 = -1
w1 = 1
b = 0.5

bias = -b * w1

weights = torch.tensor([w0, w1])
neuron = BiasNeuron(weights=weights, bias=bias, activation_function=threshold_function)

nn.second_layer.append(neuron)

In [ ]:
X = [(i/10, i/10) for i in range(11)]
X.extend([(i/10, (10-i)/10) for i in range(11)])
X.extend([(i/10, (10-i)/10 + 0.6) for i in range(11)])
X.extend([(i/10, (10-i)/10 - 0.6) for i in range(11)])
#X.extend([(i/10, i/10 + 0.2) for i in range(11)])
#X.extend([(i/10, i/10 - 0.2) for i in range(11)])
X.extend([(i/10, i/10 + 0.5) for i in range(11)])
X.extend([(i/10, i/10 - 0.5) for i in range(11)])
#print(X)
for x in X:
    out = nn.forward(torch.tensor(x))

    #print("out", out)

    if out >= 0.5:
        #print("Class C0")
        plt.scatter(x[0], x[1], s=10, c='red')

    else:
        #print("Class C1")
        plt.scatter(x[0], x[1], s=10, c='blue')

wf_x0 = [-2, -1, 0 , 1, 2]
wf_x1 = [-x0 + 0.5 for x0 in wf_x0]
plt.plot(wf_x0, wf_x1, c='blue')

wf_x0 = [-2, -1, 0 , 1, 2]
wf_x1 = [-x0 + 1.5 for x0 in wf_x0]
plt.plot(wf_x0, wf_x1, c='blue')

plt.grid(True)
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])